In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, BatchNormalization, SpatialDropout1D,
    GlobalAveragePooling1D, Dense, Multiply, Permute
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras import regularizers
from sklearn.preprocessing import LabelEncoder

import os
import numpy as np
import librosa
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Paths
ravdess_speech_path = '/Users/yathamlohithreddy/Desktop/vscodefloder /marsproject/Audio_Speech_Actors_01-24'
ravdess_song_path   =  '/Users/yathamlohithreddy/Desktop/vscodefloder /marsproject/Audio_Song_Actors_01-24'

# Settings
max_len = 300
SAMPLE_RATE = 16000
sequences = []
labels = []

# Emotion map from RAVDESS (filename part 3: 03-01-XX-...)
ravdess_emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# === Load & Preprocess ===
def load_and_preprocess(file_path, sr=SAMPLE_RATE):
    y, _ = librosa.load(file_path, sr=sr)
    y, _ = librosa.effects.trim(y, top_db=30)
    y = y / np.max(np.abs(y)) if np.max(np.abs(y)) > 0 else y
    return y

# === Feature Extraction ===
def extract_features(y, sr):
    mfcc     = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    rolloff  = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rms      = librosa.feature.rms(y=y)
    zcr      = librosa.feature.zero_crossing_rate(y=y)

    try:
        f0 = librosa.yin(y, sr=sr, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
        f0 = f0.reshape(1, -1)
    except:
        f0 = np.zeros((1, mfcc.shape[1]))

    T = mfcc.shape[1]
    def resize(f): return f[:, :T] if f.shape[1] >= T else np.pad(f, ((0, 0), (0, T - f.shape[1])))
    
    all_features = np.vstack([
        mfcc,
        resize(rolloff),
        resize(rms),
        resize(zcr),
        resize(f0)
    ])
    return all_features.T  # shape: (T, feature_dim)

# === Process RAVDESS Dataset ===
def process_ravdess(path, tag=""):
    count = 0
    for actor_folder in os.listdir(path):
        folder_path = os.path.join(path, actor_folder)
        if not os.path.isdir(folder_path):
            continue
        for file in os.listdir(folder_path):
            if not file.endswith(".wav"):
                continue
            parts = file.split("-")
            if len(parts) < 3:
                continue
            emotion_code = parts[2]
            label = ravdess_emotion_map.get(emotion_code)
            if not label:
                continue

            file_path = os.path.join(folder_path, file)
            y = load_and_preprocess(file_path)
            features = extract_features(y, SAMPLE_RATE)

            sequences.append(features)
            labels.append(label)
            count += 1
    print(f"✔️ {tag} - Loaded {count} samples")

# Process both datasets
process_ravdess(ravdess_speech_path, tag="Speech")
process_ravdess(ravdess_song_path, tag="Song")

# === Finalize ===
X = pad_sequences(sequences, maxlen=max_len, dtype='float32', padding='post', truncating='post')
le = LabelEncoder()
y = le.fit_transform(labels)

# === Summary ===
print("✅ Total samples (RAVDESS Speech + Song):", len(labels))
print("✅ X shape:", X.shape)   # (samples, max_len, features)
print("✅ y shape:", y.shape)   # (samples,)

print("\nEmotion label mapping:")
for idx, emotion in enumerate(le.classes_):
    print(f"  {idx} → {emotion}")


from sklearn.model_selection import train_test_split

# Split into train/validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Train shape:", X_train.shape)
print("Val shape:", X_val.shape)



    # suppose X.shape == (n_samples, timesteps, feat_dim)
n_samples, timesteps, feat_dim = X_train.shape

# 1) flatten
X_flat = X_train.reshape(n_samples, timesteps * feat_dim)

# 2) resample
smote = SMOTE(random_state=42)
X_flat_res, y_res = smote.fit_resample(X_flat, y_train)

# 3) reshape back to sequences
n_resampled = X_flat_res.shape[0]
X_res = X_flat_res.reshape(n_resampled, timesteps, feat_dim)

print("Resampled shapes:", X_res.shape, y_res.shape)






# -------------------------- Resample with SMOTE ----------------------------
n_samples, timesteps, feat_dim = X_train.shape
X_flat = X_train.reshape(n_samples, timesteps * feat_dim)
smote = SMOTE(random_state=42)
X_flat_res, y_res = smote.fit_resample(X_flat, y_train)
n_resampled = X_flat_res.shape[0]
X_res = X_flat_res.reshape(n_resampled, timesteps, feat_dim)

# -------------------------- Augmentations ----------------------------------
def add_noise(X, noise_level=0.01):
    return X + np.random.normal(0, noise_level, X.shape)

def random_time_scaling(X, scale_range=(0.9, 1.1)):
    return np.array([seq * np.random.uniform(*scale_range) for seq in X])

def time_mask(X, max_mask_size=5):
    X_masked = X.copy()
    for i in range(X.shape[0]):
        t = np.random.randint(0, X.shape[1] - max_mask_size)
        X_masked[i, t:t+max_mask_size, :] = 0
    return X_masked

def time_shift(X, shift_max=5):
    shifted = []
    for seq in X:
        shift = np.random.randint(-shift_max, shift_max)
        rolled = np.roll(seq, shift, axis=0)
        if shift > 0:
            rolled[:shift, :] = 0
        elif shift < 0:
            rolled[shift:, :] = 0
        shifted.append(rolled)
    return np.array(shifted)

X_aug = np.concatenate([
    X_res,
    add_noise(X_res),
    random_time_scaling(X_res),
    time_mask(X_res),
    time_shift(X_res)
], axis=0)

y_aug = np.concatenate([y_res] * 5, axis=0)

# -------------------------- Attention Layer --------------------------------
def attention_1d(inputs):
    a = Permute((2, 1))(inputs)
    a = Dense(inputs.shape[1], activation='softmax')(a)
    a = Permute((2, 1))(a)
    return Multiply()([inputs, a])

# -------------------------- Model Definition -------------------------------
num_emotions = len(np.unique(y_aug))
input_layer = Input(shape=(timesteps, feat_dim))

x = Conv1D(128, kernel_size=9, activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(1e-4))(input_layer)
x = BatchNormalization()(x)
x = SpatialDropout1D(0.3)(x)

x = Conv1D(128, kernel_size=7, activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(1e-4))(x)
x = BatchNormalization()(x)
x = SpatialDropout1D(0.3)(x)

x = Conv1D(128, kernel_size=5, activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(1e-4))(x)
x = BatchNormalization()(x)
x = SpatialDropout1D(0.3)(x)

x = Conv1D(64, kernel_size=3, activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(1e-4))(x)
x = BatchNormalization()(x)
x = SpatialDropout1D(0.3)(x)

x = attention_1d(x)
x = GlobalAveragePooling1D()(x)
output = Dense(num_emotions, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output)

# -------------------------- Compile & Train --------------------------------
lr_schedule = ExponentialDecay(1e-4, decay_steps=1000, decay_rate=0.9)
optimizer = Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

# Training
history = model.fit(
    X_aug, y_aug,
    validation_data=(X_val, y_val),
    epochs=90,
    batch_size=32,
    callbacks=[checkpoint, reduce_lr, early_stop],
    class_weight=class_weights
)

# -------------------------- Evaluation -------------------------------------
y_val_pred_probs = model.predict(X_val)
y_val_pred = np.argmax(y_val_pred_probs, axis=1)

print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:\n", classification_report(y_val, y_val_pred, target_names=le.classes_))

# -------------------------- Confusion Matrix -------------------------------
cm_val = confusion_matrix(y_val, y_val_pred)
cm_val_norm = cm_val.astype('float') / cm_val.sum(axis=1)[:, np.newaxis]
class_names = list(le.classes_)
tick_marks = np.arange(len(class_names))

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].imshow(cm_val, cmap=plt.cm.Blues)
axes[0].set_title("Confusion Matrix (Counts)")
axes[1].imshow(cm_val_norm, cmap=plt.cm.Blues)
axes[1].set_title("Confusion Matrix (Normalized)")

for ax in axes:
    ax.set_xticks(tick_marks)
    ax.set_yticks(tick_marks)
    ax.set_xticklabels(class_names, rotation=45, ha='right')
    ax.set_yticklabels(class_names)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')

for i in range(len(class_names)):
    for j in range(len(class_names)):
        axes[0].text(j, i, cm_val[i, j], ha='center', va='center',
                     color="white" if cm_val[i, j] > cm_val.max() / 2. else "black")
        axes[1].text(j, i, f"{cm_val_norm[i, j]:.2f}", ha='center', va='center',
                     color="white" if cm_val_norm[i, j] > 0.5 else "black")

plt.tight_layout()
plt.show()


✔️ Speech - Loaded 1440 samples
✔️ Song - Loaded 1012 samples
✅ Total samples (RAVDESS Speech + Song): 2452
✅ X shape: (2452, 300, 17)
✅ y shape: (2452,)

Emotion label mapping:
  0 → angry
  1 → calm
  2 → disgust
  3 → fearful
  4 → happy
  5 → neutral
  6 → sad
  7 → surprised
Train shape: (1961, 300, 17)
Val shape: (491, 300, 17)
Resampled shapes: (2408, 300, 17) (2408,)
Epoch 1/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.1250 - loss: 2.3417

377/377 ━━━━━━━━━━━━━━━━━━━━ 59s 151ms/step - accuracy: 0.1250 - loss: 2.3417 - val_accuracy: 0.1263 - val_loss: 2.1106 - learning_rate: 9.6106e-05
Epoch 2/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.2061 - loss: 2.3326

377/377 ━━━━━━━━━━━━━━━━━━━━ 74s 197ms/step - accuracy: 0.2061 - loss: 2.3326 - val_accuracy: 0.1568 - val_loss: 2.1079 - learning_rate: 9.2363e-05
Epoch 3/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 72s 191ms/step - accuracy: 0.2398 - loss: 2.3055 - val_accuracy: 0.1670 - val_loss: 2.1089 - learning_rate: 8.8766e-05
Epoch 4/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - accuracy: 0.2613 - loss: 2.2896

377/377 ━━━━━━━━━━━━━━━━━━━━ 78s 207ms/step - accuracy: 0.2613 - loss: 2.2896 - val_accuracy: 0.1833 - val_loss: 2.0994 - learning_rate: 8.5310e-05
Epoch 5/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 72s 190ms/step - accuracy: 0.2739 - loss: 2.2641 - val_accuracy: 0.1752 - val_loss: 2.1057 - learning_rate: 8.1987e-05
Epoch 6/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.3005 - loss: 2.2150

377/377 ━━━━━━━━━━━━━━━━━━━━ 72s 190ms/step - accuracy: 0.3005 - loss: 2.2150 - val_accuracy: 0.1894 - val_loss: 2.0779 - learning_rate: 7.8795e-05
Epoch 7/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.3192 - loss: 2.1810

...

377/377 ━━━━━━━━━━━━━━━━━━━━ 78s 206ms/step - accuracy: 0.4384 - loss: 1.4764 - val_accuracy: 0.2831 - val_loss: 1.8747 - learning_rate: 3.0373e-05
Epoch 31/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 78s 207ms/step - accuracy: 0.4393 - loss: 1.4639 - val_accuracy: 0.2729 - val_loss: 1.8764 - learning_rate: 2.9190e-05
Epoch 32/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 74s 196ms/step - accuracy: 0.4454 - loss: 1.4494 - val_accuracy: 0.2790 - val_loss: 1.8940 - learning_rate: 2.8053e-05
Epoch 33/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 58s 155ms/step - accuracy: 0.4401 - loss: 1.4439 - val_accuracy: 0.2790 - val_loss: 1.8809 - learning_rate: 2.6961e-05
Epoch 34/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.4373 - loss: 1.4295

377/377 ━━━━━━━━━━━━━━━━━━━━ 47s 124ms/step - accuracy: 0.4373 - loss: 1.4295 - val_accuracy: 0.2790 - val_loss: 1.8722 - learning_rate: 2.5911e-05
Epoch 35/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.4404 - loss: 1.4319

377/377 ━━━━━━━━━━━━━━━━━━━━ 69s 184ms/step - accuracy: 0.4404 - loss: 1.4319 - val_accuracy: 0.2770 - val_loss: 1.8650 - learning_rate: 2.4902e-05
Epoch 36/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 68s 181ms/step - accuracy: 0.4551 - loss: 1.4047 - val_accuracy: 0.2811 - val_loss: 1.8701 - learning_rate: 2.3932e-05
Epoch 37/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 81s 177ms/step - accuracy: 0.4435 - loss: 1.4061 - val_accuracy: 0.2729 - val_loss: 1.8751 - learning_rate: 2.3000e-05
Epoch 38/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 75s 200ms/step - accuracy: 0.4385 - loss: 1.4135 - val_accuracy: 0.2770 - val_loss: 1.8688 - learning_rate: 2.2104e-05
Epoch 39/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 56s 148ms/step - accuracy: 0.4513 - loss: 1.3837 - val_accuracy: 0.2831 - val_loss: 1.8713 - learning_rate: 2.1244e-05
Epoch 40/90
376/377 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.4508 - loss: 1.3647

377/377 ━━━━━━━━━━━━━━━━━━━━ 42s 110ms/step - accuracy: 0.4508 - loss: 1.3647 - val_accuracy: 0.2851 - val_loss: 1.8523 - learning_rate: 2.0416e-05
Epoch 41/90
377/377 ━━━━━━━━━━━━━━━━━━━━ 47s 125ms/step - accuracy: 0.4528 - loss: 1.3617 - val_accuracy: 0.2811 - val_loss: 1.8677 - learning_rate: 1.9621e-05
Epoch 42/90
101/377 ━━━━━━━━━━━━━━━━━━━━ 31s 113ms/step - accuracy: 0.4578 - loss: 1.3441

KeyboardInterrupt: 